# Workshop: Machine learning with text in Scikit-learn
## Outline

1. [Introduction to supervised learning in scikit-learn](#Introduction-to-supervised-learning-in-scikit-learn)
1. [Converting text to feature vectors](#From-text-to-feature-vectors)
1. [Classifying creditors from the Czech Insolvency Register](#Classifying-creditors-from-the-Czech-Insolvency-Register)
    1. [Loading and preprocessing the dataset](#Loading-and-preprocessing-the-dataset)
    1. [Vectorizing the dataset](#Vectorizing-the-dataset)
    1. [Building and evaluating the model](#Building-and-evaluating-the-model)
    1. [Examining the model](#Examining-the-model)
1. [Topics not covered](#Topics-not-covered)

In [1]:
import pandas as pd
import unicodedata
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [42]:
# for Python 2: use print only as a function
from __future__ import print_function

## Introduction to supervised learning in scikit-learn

**From <a href="https://en.wikipedia.org/wiki/Supervised_learning">Wikipedia</a>:**<br>
**Supervised learning** is the machine learning task of inferring a function from labeled training data. The training data consist of a set of training examples. In supervised learning, each example is a pair consisting of an input object (typically a vector) and a desired output value.

**Note:** We will consider a classification task, i.e., samples belong to two or more classes that we want to predict.

In [2]:
# Load the iris dataset.
from sklearn.datasets import load_iris
iris = load_iris()

In [3]:
X = iris.data      # feature matrix, on sample per row
y = iris.target    # target vector

In [4]:
# Let's examine the shapes of X and y.
print(X.shape)
print(y.shape)

(150, 4)
(150,)


In [5]:
n_features = X.shape[1]
n_features

4

In [6]:
# Nicer overview of our dataset.
dataset = pd.DataFrame(X, columns=iris.feature_names)
dataset["label"] = y
dataset.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
# Let's examine the target vector
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [8]:
# Init logistic regression model with default params.
clf = LogisticRegression()

# Fit the model. 
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
# Take a sample from the training data.
s = X[0]
s

array([ 5.1,  3.5,  1.4,  0.2])

In [10]:
# Let's try to predict the target value for this sample.
clf.predict([s])

array([0])

**Note:** In scikit-learn, an estimator for classification is a Python object that implements the methods **fit(X, y)** and **predict(samples)**. 

**To summarize the general process:**
1. Get a dataset in form **X** (feature matrix) and **y** (target variable)
2. Pick a model and fit it using **.fit(X, y)**
3. Predict values of new, unobserved samples using **.predict(samples)**

You can also check the basic introduction to ML with scikit-learn in the <a href="http://scikit-learn.org/stable/tutorial/basic/tutorial.html">documentation</a>.

## From text to feature vectors

In [11]:
text_dataset = ["A coward judges all he sees by what he is.",
                "There are people who need people to need them.",
                "Never's the word God listens for when he needs a laugh."]

### Problem

From <a target="_blank" href="http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction">scikit-learn documentation</a>:<br>
Text Analysis is a major application field for machine learning algorithms. However the raw data, **a sequence of symbols cannot be fed directly to the algorithms themselves** as most of them expect **numerical feature vectors with a fixed size** rather than the **raw text documents with variable length**.

### Solution

From <a target="_blank" href="http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html">scikit-learn documentation</a>:<br>

In order to perform machine learning on text documents, we first need to turn the text content into **numerical feature vectors**.

The most intuitive way to do so is the **bags of words** representation:
1. assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices).
1. for each document **#i**, count the number of occurrences of each word w and store it in **X[i, j]** as the value of feature **#j** where **j** is the index of word **w** in the dictionary

The bags of words representation implies that **n_features** is the number of distinct words in the corpus: **this number is typically larger than 100,000**.

Fortunately, **most values in X will be zeros** since for a given document less than a couple thousands of distinct words will be used. For this reason we say that bags of words are typically high-dimensional sparse datasets. We can save a lot of memory by only storing the non-zero parts of the feature vectors in memory.


We will use scikits **CountVectorizer** to convert text into a **matrix of token counts (document-term matrix)**:

In [12]:
# Init CountVectorizer with the default params.
vectorizer = CountVectorizer()

In [13]:
# Learn the vocabulary from the text data.
vectorizer.fit(text_dataset)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [14]:
# Examine the vocabulary.
vocabulary = vectorizer.get_feature_names()
print("Vocabulary size: {0}".format(len(vocabulary)))
print("Vocabulary:")
print(vocabulary)

Vocabulary size: 24
Vocabulary:
[u'all', u'are', u'by', u'coward', u'for', u'god', u'he', u'is', u'judges', u'laugh', u'listens', u'need', u'needs', u'never', u'people', u'sees', u'the', u'them', u'there', u'to', u'what', u'when', u'who', u'word']


In [15]:
# Transform text data into a document-term matrix.
dtm = vectorizer.transform(text_dataset)
dtm

<3x24 sparse matrix of type '<type 'numpy.int64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [16]:
# Let's examine the obtained document-term matrix.
pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names())

,all,are,by,coward,for,god,he,is,judges,laugh,...,people,sees,the,them,there,to,what,when,who,word
0,1,0,1,1,0,0,2,1,1,0,...,0,1,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,2,0,0,1,1,1,0,0,1,0
2,0,0,0,0,1,1,1,0,0,1,...,0,0,1,0,0,0,0,1,0,1


**Summary:**<br>
**Vectorization** is a general process of turning a collection of text documents into numerical feature vectors.<br>
**CountVectorizer** is one of the vectorizers available in scikit-learn.<br>
All vectorizers are used as follows:
* use **.fit(data)** to learn the vocabulary
* use **.transform(data)** to build the document-term matrix from text data

## Classifying creditors from the Czech Insolvency Register

### Loading and preprocessing the dataset

In [17]:
# Load the dataset with pandas.
dataset = pd.read_table("./data/receivables.tsv", encoding="utf-8", header=0)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 3 columns):
documentId    3500 non-null int64
creditor      3500 non-null object
text          3500 non-null object
dtypes: int64(1), object(2)
memory usage: 82.1+ KB


In [18]:
# Check the dataset.
dataset.head()

,documentId,creditor,text
0,3771532,Cetelem CR,přihláška pohledávky sülldí krajský soud v brn...
1,435902,Cetelem CR,krajsky soud v. ceskych nugajuariaau _ -i;i5;'...
2,4354526,Cetelem CR,liřlhll-íšiíá puhledáxřke” krajský soud v ústi...
3,46081,Cetelem CR,prihlaska pohledaefkt' 501152 krajsky soud v c...
4,4764086,Cetelem CR,přihláška pohledávky soud: krajský soud v plzn...


In [19]:
# Check the number of samples per class.
dataset.groupby('creditor').count()

,documentId,text
creditor,,
CEZ Prodej,250,250
CSOB,250,250
Ceska sporitelna,250,250
Cetelem CR,250,250
Cofidis,250,250
Essox,250,250
GE Money Bank,250,250
Home Credit,250,250
Komercni banka,250,250


In [20]:
# Since we have texts written in Czech in the dataset, let's remove the accents (diacritics) from the text first.
def remove_accents(s):
    nkfd_form = unicodedata.normalize('NFKD', s)
    ascii_string = nkfd_form.encode('ASCII', 'ignore')
    return ascii_string

dataset["text"] = dataset["text"].apply(remove_accents)

In [21]:
# Check the dataset without accents again.
dataset.head()

,documentId,creditor,text
0,3771532,Cetelem CR,prihlaska pohledavky sulldi krajsky soud v brn...
1,435902,Cetelem CR,krajsky soud v. ceskych nugajuariaau _ -i;i5;'...
2,4354526,Cetelem CR,lirlhll-isiia puhledaxrke krajsky soud v usti ...
3,46081,Cetelem CR,prihlaska pohledaefkt' 501152 krajsky soud v c...
4,4764086,Cetelem CR,prihlaska pohledavky soud: krajsky soud v plzn...


In [22]:
# scikit-learn required numerical values as labels, so let's convert 
# the creditors' names to numbers first using LabelEncoder.
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(dataset['creditor'])
numeric_labels

array([ 3,  3,  3, ..., 12, 12, 12])

In [23]:
# Check the classes.
label_encoder.classes_

array([u'CEZ Prodej', u'CSOB', u'Ceska sporitelna', u'Cetelem CR',
       u'Cofidis', u'Essox', u'GE Money Bank', u'Home Credit',
       u'Komercni banka', u'Profidebt', u'T-Mobile CR', u'VZP', u'unknown'], dtype=object)

In [24]:
# Example usage of label_encoder.
label_encoder.transform(["CSOB"])

array([1])

In [25]:
# Add the numberic_labels to the dataset so that we have everything in one place.
dataset["numeric_label"] = numeric_labels
dataset.head()

,documentId,creditor,text,numeric_label
0,3771532,Cetelem CR,prihlaska pohledavky sulldi krajsky soud v brn...,3
1,435902,Cetelem CR,krajsky soud v. ceskych nugajuariaau _ -i;i5;'...,3
2,4354526,Cetelem CR,lirlhll-isiia puhledaxrke krajsky soud v usti ...,3
3,46081,Cetelem CR,prihlaska pohledaefkt' 501152 krajsky soud v c...,3
4,4764086,Cetelem CR,prihlaska pohledavky soud: krajsky soud v plzn...,3


In [26]:
# Get the feature vectors and target variables.
# The feature vector still contains just raw texts.
X = dataset["text"]
y = dataset["numeric_label"]
print(X.shape)
print(y.shape)

(3500,)
(3500,)


In [27]:
# Split X and y into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2625,)
(875,)
(2625,)
(875,)


### Vectorizing the dataset

In [28]:
# Load the prepared list of stopwords for the Czech language.
stopwords = pd.read_csv('data/stopwords_cz.txt', encoding='utf-8', header=None, names=["word"])
print("Number of stopwords: {0}".format(len(stopwords)))
stopwords.head()

Number of stopwords: 256


,word
0,ačkoli
1,ahoj
2,ale
3,anebo
4,ano


In [29]:
# Initialize the CountVectorizer, this time with customized params.
vectorizer = CountVectorizer(lowercase=True,
                             ngram_range=(1,3),
                             stop_words=list(stopwords["word"].values),
                             max_df = 0.5,
                             min_df = 30,
                             tokenizer = lambda x: re.split("[\r\t\n .,;:'\"()?!/]+", x))



**Parameters:**
* **lowercase** - convert all characters to lowercase before tokenizing.
* **ngram_range** - the lower and upper boundary of the range of n-values for different n-grams to be extracted. 
* **stop_words** - list of stopwords which will be removed from the vocabulary.
* **max_df** - ignore terms that have a document frequency strictly higher than the this threshold (float from [0.0, 1.0] for relative value or integer for absolute value).
* **min_df** - ignore terms that have a document frequency strictly lower than the this threshold (float from [0.0, 1.0] for relative value or integer for absolute value).
* **tokenizer** - used to specify a custom tokenization (i.e. splitting text to words) step.

In [30]:
# Learn the vocabulary and check its size.
vectorizer.fit(X_train)
len(vectorizer.get_feature_names())

13834

In [31]:
# Transform train data into a document-term matrix.
X_train_dtm = vectorizer.transform(X_train)
X_train_dtm

<2625x13834 sparse matrix of type '<type 'numpy.int64'>'
	with 1874035 stored elements in Compressed Sparse Row format>

In [32]:
# Transform test data into a document-term matrix.
X_test_dtm = vectorizer.transform(X_test)
X_test_dtm

<875x13834 sparse matrix of type '<type 'numpy.int64'>'
	with 632708 stored elements in Compressed Sparse Row format>

### Building and evaluating the model

In [33]:
# Init logistic regression model, this time with slightly changed params.
clf = LogisticRegression(C=1.0, penalty='l1')

In [34]:
# Train the model and time it with IPython magic command.
%time clf.fit(X_train_dtm, y_train)

CPU times: user 2.34 s, sys: 8 ms, total: 2.35 s
Wall time: 2.35 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
# Make predictions for test data.
y_predictions = clf.predict(X_test_dtm)

In [36]:
# Calculate the accuracy of your predictions.
metrics.accuracy_score(y_test, y_predictions)

0.94628571428571429

In [37]:
# Print the confusion matrix.
pd.DataFrame(metrics.confusion_matrix(y_test, y_predictions), 
             index=label_encoder.classes_, columns=label_encoder.classes_)

,CEZ Prodej,CSOB,Ceska sporitelna,Cetelem CR,Cofidis,Essox,GE Money Bank,Home Credit,Komercni banka,Profidebt,T-Mobile CR,VZP,unknown
CEZ Prodej,66,0,1,0,0,0,0,0,0,0,0,0,1
CSOB,0,66,1,0,0,0,0,0,1,0,1,0,2
Ceska sporitelna,0,0,54,1,0,0,0,2,1,0,0,0,3
Cetelem CR,0,0,0,62,0,0,0,0,0,0,0,0,0
Cofidis,0,0,0,0,59,0,0,0,0,0,0,0,5
Essox,0,0,0,0,0,56,0,0,0,0,0,0,0
GE Money Bank,0,0,0,0,0,0,73,0,0,0,0,0,0
Home Credit,0,0,0,1,0,0,0,62,0,1,0,0,3
Komercni banka,0,0,0,0,0,0,0,0,52,0,0,0,5
Profidebt,0,0,0,1,0,0,0,0,0,56,0,0,0


### Examining the model
Can we somehow find out what has the model actually learned?

In [38]:
# Check the logistic regressions params.
print(clf.coef_.shape)
print(clf.coef_)

(13, 13834)
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.         -0.01095226  0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [-0.11066957  0.          0.         ...,  0.          0.          0.        ]]


In [39]:
# Get list of feature names and classes.
feature_names = vectorizer.get_feature_names()
classes = label_encoder.classes_

In [40]:
# Create a redable version of our model and check its learned parameters.
readable_model = pd.DataFrame(clf.coef_.transpose(), columns=classes)
readable_model.insert(0, "ngram", feature_names)
readable_model[7000:8000]

,ngram,CEZ Prodej,CSOB,Ceska sporitelna,Cetelem CR,Cofidis,Essox,GE Money Bank,Home Credit,Komercni banka,Profidebt,T-Mobile CR,VZP,unknown
7000,lkrajskysoudvbrne,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7001,lkrajskysoudvplzni,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7002,lkspl,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7003,ll,0.0,0.0,0.000000,0.0,0.0,0.0,0.162669,-0.402833,0.0,0.0,0.0,0.0,0.000000
7004,ll dalsi,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7005,ll dalsi okolnosti,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7006,ll l,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7007,ll pohledavka,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7008,ll-,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
7009,lli,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000


In [41]:
# Examine parameters for specific class.
readable_model.sort_values(by="Ceska sporitelna", ascending=False).head(10)

,ngram,CEZ Prodej,CSOB,Ceska sporitelna,Cetelem CR,Cofidis,Essox,GE Money Bank,Home Credit,Komercni banka,Profidebt,T-Mobile CR,VZP,unknown
11257,sporitelna,0.0,0.0,3.434089,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1832,1929,0.0,0.0,2.123530,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3790,autorita,0.0,0.0,1.728772,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1721,14000,0.0,0.0,1.345415,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8100,olbrachtova,0.0,0.0,1.199837,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,-0.298917
7112,ma,0.0,0.0,1.015765,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1032,-_ _,0.0,0.0,0.972186,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
10754,sds,0.0,0.0,0.829416,0.000000,0.00000,0.0,0.0,-0.004357,0.000000,0.0,0.000000,0.000000,0.000000
12184,uver,0.0,0.0,0.799339,0.000000,0.00000,0.0,0.0,0.192864,0.000000,0.0,0.000000,0.000000,-0.436198
9434,praha 4,0.0,0.0,0.762380,-0.648322,-0.18885,0.0,0.0,-1.226415,-0.612462,0.0,0.627737,-0.446886,-0.079370


What has the model learned? Check
<a target="_blank" href="https://www.google.cz/maps/@50.0448653,14.4483488,3a,75y,195.68h,98.1t/data=!3m6!1e1!3m4!1s0K232cRC0B4x6gwNN8GG7Q!2e0!7i13312!8i6656!6m1!1e1?hl=en">Olbrachtova 1929, Praha 4</a>.

## Topics not covered
* Advanced text preprocessing.
* Better model evaluation.
* Comparison of different models.
* (Meta) Parameter optimization of each model.
* ... and much much more.

# Dataset preparation [TO DELETE]

In [ ]:
data = pd.read_table("./data/receivables.tsv", encoding="utf-8", header=0)

In [ ]:
def normalize_whitespace(s):
    return re.sub(r'\s+', ' ', s.strip())

In [ ]:
creditors = {
    u"GE Money Bank":"25672720",
    u"Česká spořitelna":"45244782",
    u"Cetelem ČR":"25085689",
    u"Provident Financial":"25621351",
    u"Profi Credit Czech":"61860069",
    u"Smart Capital":"26865297",
    u"Home Credit":"26978636",
    u"Essox":"26764652",
    u"Telefónica CR":"60193336",
    u"T-Mobile CR":"64949681",
    u"VZP":"41197518",
    u"Komerční banka":"45317054",
    u"ČEZ Prodej":"27232433",
    u"Cofidis":"27179907",
    u"ČSOB":"1350",
    u"Profidebt":"27221971",
    u"Citibank Europe":"28198131",
    u"Raiffeisenbank":"49240901",
    u"Česká podnikatelská pojišťovna":"63998530",
    u"Kooperativa pojišťovna":"47116617",
    u"Bohemia Faktoring":"27242617"
}
creditors_inverted = dict([(val, key) for key,val in creditors.items()])
data["creditor"] = data["ico"].apply(lambda x: remove_accents(creditors_inverted.get(x, u"unknown")))
data["documentId"] = data["documentId"].astype(int)
data["text"] = data["text"].apply(lambda text: normalize_whitespace(re.sub(u'[\x00-\x20]+', ' ', text))[:10000])



In [ ]:
data_uknown = data[data["creditor"] == "unknown"]

In [ ]:
data = data[data["creditor"].apply(lambda creditor: creditor not in [u"Bohemia Faktoring", 
                                                                 u'Profi Credit Czech', 
                                                                 u'Citibank Europe', u'Smart Capital', u'Telefonica CR',
                                                                 u'Provident Financial', u'Raiffeisenbank', 
                                                                 u'unknown'])]

In [ ]:
data = data.groupby('creditor').head(250).append(data_uknown[:500])

In [ ]:
data[["documentId", "creditor", "text"]].to_csv("./data/receivables_new.tsv", sep="\t", encoding="utf-8", index=False, header=True)

In [ ]:
data["text_len"] = data["text"].apply(lambda text: len(text))

In [ ]:
data